# Import Packages

In [12]:
# -*- coding: utf-8 -*-
"""
Created on April 4, 2024

First Version Author: George Zhang


"""
User_Ackowledgement = input("If you are not the first author, please ADD comments with your name and date for ALL your edits. Ackowledge by answering Y", )

#if User_Ackowledgement is not Y, raise error

if User_Ackowledgement != 'Y':

    raise ValueError("You must acknowledge your edits by answering Y. Please add your name and date in the comments for your edits.")


#%%
from IPython import get_ipython
get_ipython().magic('reset -sf')

import sys
 
# appending a path
sys.path.append('')
 
import os
import pandas as pd
import numpy as np
import pickle
# import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt
import math
import pyodbc

C:\Users\georgez\AppData\Local\Temp\ipykernel_20260\3988292044.py:20: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


# Setting Directories

In [13]:
current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added

input_dir = code_dir + '\\..\\' + "Inputs" #JT
Output_dir = code_dir + '\\..\\' + "Outputs" #JT

Jason_Input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs"

Jason_Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs"


# Input and Proces Control

# Compare EQB Dec. 2022 with Dec.2024

In [3]:
# Read eqb_rwa_original_2024 parquet from Outputs

eqb_rwa_original_202412 = pd.read_parquet(Output_dir + '\\eqb_rwa_original.parquet', engine='pyarrow')

# Read eqb_rwa_original_202212 parquet from Jason Outputs

eqb_rwa_original_202212 = pd.read_parquet(Jason_Output_dir + '\\eqb_rwa_original.parquet', engine='pyarrow')

In [4]:
# Add a column Reporting_Date to eqb_rwa_original_202412 to have value '2024-12-31'

eqb_rwa_original_202412['Reporting_Date'] = pd.to_datetime('2024-12-31')

# Add a column Reporting_Date to eqb_rwa_original_202212 to have value '2022-12-31'

eqb_rwa_original_202212['Reporting_Date'] = pd.to_datetime('2022-12-31')



In [ ]:
# Concatenate eqb_rwa_original_202412 and eqb_rwa_original_202212

SFR_rwa_2022_2024 = pd.concat([eqb_rwa_original_202412, eqb_rwa_original_202212], ignore_index=True)


In [ ]:

# Save the data set to a csv file

SFR_rwa_2022_2024.to_csv(Output_dir + '\\SFR_rwa_2022_2024.csv', index=False)



# Pivot Table for Dec. 2022 vs. Dec. 2024

In [6]:
# Merge eqb_rwa_original_202412 and eqb_rwa_original_202212 on 'Loan_Number' and keep identifiers from both dataframes

df_Merge = pd.merge(eqb_rwa_original_202412, eqb_rwa_original_202212, on='Loan_Number', how='outer', suffixes=('_2024', '_2022'), indicator=True)


In [7]:
# Create a pivot table with rows as MRS_Bin_PD_2022 and columns as MRS_Bin_PD_2024, and values as count of Loan_Number

# Fill missing values in MRS_Bin_PD_2024 with 'Closed'

df_Merge['MRS_Bin_PD_2024'] = df_Merge['MRS_Bin_PD_2024'].fillna('Closed')

# Fill missing values in MRS_Bin_PD_2022 with 'New'

df_Merge['MRS_Bin_PD_2022'] = df_Merge['MRS_Bin_PD_2022'].fillna('New')

pivot_table = pd.pivot_table(df_Merge,
                             index='MRS_Bin_PD_2022',
                             columns='MRS_Bin_PD_2024',
                             values='Loan_Number',
                             aggfunc='count',
                                fill_value=0, 
                                margins=True,
                                margins_name='Total',
                                dropna=False)




# Compare Concentra Dec. 2022 with Dec. 2024

In [8]:
# Read 'concentra_rwa_addon-pkl' from output directory

concentra_rwa_addon_202412 = pd.read_pickle(Output_dir + '\\concentra_rwa_addon.pkl')

# Read 'concentra_rwa_addon-pkl' from Jason output directory

concentra_rwa_addon_202212 = pd.read_pickle(Jason_Output_dir + '\\concentra_rwa_addon.pkl')


In [9]:
# Add a column Reporting_Date to concentra_rwa_addon_202412 to have value '2024-12-31'

concentra_rwa_addon_202412['Reporting_Date'] = pd.to_datetime('2024-12-31')

# Add a column Reporting_Date to concentra_rwa_addon_202212 to have value '2022-12-31'

concentra_rwa_addon_202212['Reporting_Date'] = pd.to_datetime('2022-12-31')

In [ ]:
# Concatenate concentra_rwa_addon_202412 and concentra_rwa_addon_202212

concentra_rwa_addon_2022_2024 = pd.concat([concentra_rwa_addon_202412, concentra_rwa_addon_202212], ignore_index=True)


In [ ]:

# Save the data set to a csv file

concentra_rwa_addon_2022_2024.to_csv(Output_dir + '\\concentra_rwa_addon_2022_2024.csv', index=False)

# Compare EQB Corp for Dec. 2022 vs. Dec. 2024

In [14]:
# Read EQB_Corp_rwa_addon.pkl from Jason output directory

eqb_corp_rwa_addon_202212 = pd.read_pickle(Jason_Output_dir + '\\EQB_Corp_rwa_addon.pkl')

# Read EQB_Corp_rwa_addon.pkl from output directory

eqb_corp_rwa_addon_202412 = pd.read_pickle(Output_dir + '\\EQB_Corp_rwa_addon.pkl')

In [15]:
# Add a column Reporting_Date to eqb_corp_rwa_addon_202412 to have value '2024-12-31'

eqb_corp_rwa_addon_202412['Reporting_Date'] = pd.to_datetime('2024-12-31')

# Add a column Reporting_Date to eqb_corp_rwa_addon_202212 to have value '2022-12-31'

eqb_corp_rwa_addon_202212['Reporting_Date'] = pd.to_datetime('2022-12-31')


In [16]:
# Concatenate eqb_corp_rwa_addon_202412 and eqb_corp_rwa_addon_202212

eqb_corp_rwa_addon_2022_2024 = pd.concat([eqb_corp_rwa_addon_202412, eqb_corp_rwa_addon_202212], ignore_index=True)


In [17]:
# Save the data set to a csv file

eqb_corp_rwa_addon_2022_2024.to_csv(Output_dir + '\\eqb_corp_rwa_addon_2022_2024.csv', index=False)

# Double-Check Outputs

In [11]:
# Read SFR_rwa_2022_2024.csv from output_dir

SFR_rwa_2022_2024 = pd.read_csv(Output_dir + '\\SFR_rwa_2022_2024.csv')

# Group by Reporting_Date, calculate sum of RWA_new and EAD

SFR_rwa_2022_2024_grouped = SFR_rwa_2022_2024.groupby(['Reporting_Date']).agg({'RWA_new': 'sum', 'EAD': 'sum'}).reset_index()

# Calculate RWA_new / EAD

SFR_rwa_2022_2024_grouped['RWA_new_EAD'] = SFR_rwa_2022_2024_grouped['RWA_new'] / SFR_rwa_2022_2024_grouped['EAD']

print(SFR_rwa_2022_2024_grouped)


  Reporting_Date       RWA_new           EAD  RWA_new_EAD
0     2022-12-31  3.837324e+09  2.366801e+10     0.162131
1     2024-12-31  3.813415e+09  2.227326e+10     0.171210
